# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

In [2]:
import os

os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

os.environ['FABRIC_BASTION_USERNAME']='pruth'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

In [15]:
import os
import json
import traceback
from getpass import getpass

os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']='/home/mono/Sachen/Work/FABRIC/fabric_python_api_locally/tokens.json'

os.environ['FABRIC_BASTION_USERNAME']='minawm_0041350787'
os.environ['FABRIC_BASTION_KEY_LOCATION']='/home/mono/Sachen/Work/FABRIC/mina_bastion_key'

os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']="/home/mono/.ssh/id_rsa" #'/home/mono/Sachen/Work/FABRIC/mina_bastion_key' #os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']="/home/mono/.ssh/id_rsa.pub" #'/home/mono/Sachen/Work/FABRIC/mina_bastion_key.pub' #os.environ['HOME']+'/.ssh/id_rsa.pub'
print('Please input private key passphrase. Press enter for no passphrase.')
os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

Please input private key passphrase. Press enter for no passphrase.
········


## Basic FABRIC Slice Configuration

In [16]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [124]:
# Slice 
slice_name = 'P4Lang_Tutorial'

# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 40

# Hosts
h1_name = "h1"
h2_name = "h2"
h3_name = "h3"

h1_ip="10.0.1.1"
h2_ip="10.0.2.2"
h3_ip="10.0.3.3"

host_cores = 2
host_ram = 8
host_disk = 10
# Sites
site_1 = 'MAX'
site_2 = 'MAX'
site_3 = 'MAX' #'NCSA' #'STAR' #'TACC' #'STAR' #'NCSA' #'UTAH'

net_h1_name = 'net_h1'
net_h2_name = 'net_h2'
net_h3_name = 'net_h3'

net_s1_s2_name = 'net_s1_s2'
net_s2_s3_name = 'net_s2_s3'
net_s1_s3_name = 'net_s1_s3'

# All node properties
#username = 'ubuntu'
image = 'default_ubuntu_20'
vlan = '1000'
#image_type = 'qcow2'



### Create the Slice

In [125]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=site_1)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_image(image)
    [s1_iface_local] = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()
    [s1_iface_to_s2, s1_iface_to_s3] = s1.add_component(model='NIC_ConnectX_5', name="s1_switch_nic").get_interfaces()
    s1_iface_to_s2.set_vlan(vlan=vlan)
    s1_iface_to_s3.set_vlan(vlan=vlan)

    # Add switch node s2
    s2 = slice.add_node(name=s2_name, site=site_2)
    s2.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s2.set_image(image)
    [s2_iface_local] = s2.add_component(model='NIC_Basic', name="s2_local_nic").get_interfaces()
    [s2_iface_to_s1, s2_iface_to_s3] = s2.add_component(model='NIC_ConnectX_5', name="s2_switch_nic").get_interfaces()
    s2_iface_to_s1.set_vlan(vlan=vlan)
    s2_iface_to_s3.set_vlan(vlan=vlan)
   
    # Add switch node s3
    s3 = slice.add_node(name=s3_name, site=site_3)
    s3.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s3.set_image(image)
    [s3_iface_local] = s3.add_component(model='NIC_Basic', name="s3_local_nic").get_interfaces()
    [s3_iface_to_s1, s3_iface_to_s2] = s3.add_component(model='NIC_ConnectX_5', name="s3_switch_nic").get_interfaces()
    s3_iface_to_s1.set_vlan(vlan=vlan)
    s3_iface_to_s2.set_vlan(vlan=vlan)
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site_1)
    h1.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h1.set_image(image)
    [h1_iface] = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site_2)
    h2.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h2.set_image(image)
    [h2_iface] = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()
 
    # Add host node h3
    h3 = slice.add_node(name=h3_name, site=site_3)
    h3.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h3.set_image(image)
    [h3_iface] = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()
    
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[s1_iface_to_s2, s2_iface_to_s1])
    swtich_net2 = slice.add_l2network(name=net_s2_s3_name, interfaces=[s2_iface_to_s3, s3_iface_to_s2])
    swtich_net3 = slice.add_l2network(name=net_s1_s3_name, interfaces=[s3_iface_to_s1, s1_iface_to_s3])

    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_iface_local, h1_iface])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s2_iface_local, h2_iface])
    host_net3 = slice.add_l2network(name=net_h3_name, interfaces=[s3_iface_local, h3_iface])    
    
    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

Waiting for slice ................... Slice state: StableOK
Running post boot config ...Done!


# timeout needs to be extended

## Get the Slice

In [126]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           Bandwidth           : {interface.get_bandwidth()}")
            print(f"           VLAN                : {interface.get_vlan()}")  
            print(f"           MAC                 : {interface.get_mac()}") 
            print(f"           OS iface name       : {interface.get_os_interface()}")
    for network in slice.get_l2networks():
        print("Network:")
        print(f"    Name:            {network.get_name()}")
    print(f"Interface Map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : s1
   Cores             : 2
   RAM               : 8
   Disk              : 100
   Image             : default_ubuntu_20
   Image Type        : qcow2
   Host              : max-w4.fabric-testbed.net
   Site              : MAX
   Management IP     : 63.239.135.76
   Reservation ID    : 76a9a7cc-710a-43f1-816e-b9d0862e3087
   Reservation State : Active
   SSH Command       : ssh -i /home/mono/.ssh/id_rsa -J minawm_0041350787@bastion-1.fabric-testbed.net ubuntu@63.239.135.76
   Components        :  
      Name             : s1-s1_local_nic
      Details          : Mellanox ConnectX-6 VPI MCX653 dual port 100Gbps
      Disk (G)         : 0
      Units            : 1
      PCI Address      : 0000:e2:09.4
      Model            : ConnectX-6
      Type             : SharedNIC
      Name             : s1-s1_switch_nic
      Details          : Mellanox ConnectX-5 Dual Port 10/25GbE
      Disk (G)         : 0
      Units            : 1
      PCI Address      : ['0000

           MAC                 : 16:d3:e9:1f:62:72
           OS iface name       : ens7
Network:
    Name:            net_s1_s2
Network:
    Name:            net_s2_s3
Network:
    Name:            net_s1_s3
Network:
    Name:            net_h1
Network:
    Name:            net_h2
Network:
    Name:            net_h3
Interface Map: {'net_s1_s2': {'s1': {'ifname': 'ens7', 'mac': '04:3f:72:fa:78:00'}, 's2': {'ifname': 'ens7', 'mac': '04:3f:72:fa:75:60'}}, 'net_s2_s3': {'s2': {'ifname': 'ens8', 'mac': '04:3f:72:fa:75:61'}, 's3': {'ifname': 'ens8', 'mac': '04:3f:72:fa:75:5d'}}, 'net_s1_s3': {'s1': {'ifname': 'ens8', 'mac': '04:3f:72:fa:78:01'}, 's3': {'ifname': 'ens7', 'mac': '04:3f:72:fa:75:5c'}}, 'net_h1': {'s1': {'ifname': 'ens9', 'mac': '0a:6f:b0:40:4b:14'}, 'h1': {'ifname': 'ens7', 'mac': '16:55:bb:75:01:1a'}}, 'net_h2': {'s2': {'ifname': 'ens9', 'mac': '0e:b8:1e:41:bb:2e'}, 'h2': {'ifname': 'ens7', 'mac': '16:8c:37:7e:fa:3d'}}, 'net_h3': {'s3': {'ifname': 'ens9', 'mac': '02:1a:9d:93

## Configure Nodes


In [127]:
host_config_script='sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git'

try:
    h1 = slice.get_node(name=h1_name)        
    h1_os_iface = h1.get_interface(network_name=net_h1_name)  
#     iface1.set_ip(ip=h1_ip, cidr="24")
    h1_os_iface.set_ip(ip=h1_ip, cidr="24")
    
    stdout, stderr = h1.execute(host_config_script)
    print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 
    
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=net_h2_name)
    h2_os_iface.set_ip(ip=h2_ip, cidr="24")
    
    
    stdout, stderr = h2.execute(host_config_script)
    print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 
    
try:
    h3 = slice.get_node(name=h3_name)
    h3_os_iface = h3.get_interface(network_name=net_h3_name)
    h3_os_iface.set_ip(ip=h3_ip, cidr="24")
    
    stdout, stderr = h3.execute(host_config_script)
    print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 
    


stdout: Selecting previously unselected package python3-backcall.
(Reading database ... 63512 files and directories currently installed.)
Preparing to unpack .../00-python3-backcall_0.1.0-2_all.deb ...
Unpacking python3-backcall (0.1.0-2) ...
Selecting previously unselected package python3-decorator.
Preparing to unpack .../01-python3-decorator_4.4.2-0ubuntu1_all.deb ...
Unpacking python3-decorator (4.4.2-0ubuntu1) ...
Selecting previously unselected package python3-parso.
Preparing to unpack .../02-python3-parso_0.5.2-1ubuntu1_all.deb ...
Unpacking python3-parso (0.5.2-1ubuntu1) ...
Selecting previously unselected package python3-jedi.
Preparing to unpack .../03-python3-jedi_0.15.2-1_all.deb ...
Unpacking python3-jedi (0.15.2-1) ...
Selecting previously unselected package python3-pickleshare.
Preparing to unpack .../04-python3-pickleshare_0.7.5-2_all.deb ...
Unpacking python3-pickleshare (0.7.5-2) ...
Selecting previously unselected package python3-wcwidth.
Preparing to unpack .../05-

## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



In [128]:
try:
    s1 = slice.get_node(name=s1_name)
    s1_h1_os_iface = s1.get_interface(network_name=net_h1_name)
    s1_s2_os_iface = s1.get_interface(network_name=net_s1_s2_name)
    s1_s3_os_iface = s1.get_interface(network_name=net_s1_s3_name)
    
    file_attributes = s1.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    stdout = s1.execute(f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {s1_h1_os_iface.get_os_interface()} {s1_s2_os_iface.get_os_interface()} {s1_s3_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'")
    print("stdout: {}".format(stdout))

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

stdout: ('', '')


In [129]:
try:
    s2 = slice.get_node(name=s2_name)
    s2_h2_os_iface = s2.get_interface(network_name=net_h2_name)
    s2_s1_os_iface = s2.get_interface(network_name=net_s1_s2_name)
    s2_s3_os_iface = s2.get_interface(network_name=net_s2_s3_name)
    
    file_attributes = s2.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    stdout = s2.execute(f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {s2_h2_os_iface.get_os_interface()} {s2_s1_os_iface.get_os_interface()} {s2_s3_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'")
    print("stdout: {}".format(stdout))
    
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

stdout: ('', '')


In [130]:
try:
    s3 = slice.get_node(name=s3_name)
    s3_h3_os_iface = s3.get_interface(network_name=net_h3_name)
    s3_s1_os_iface = s3.get_interface(network_name=net_s1_s3_name)
    s3_s2_os_iface = s3.get_interface(network_name=net_s2_s3_name)
     
    file_attributes = s3.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    stdout = s3.execute(f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {s3_h3_os_iface.get_os_interface()} {s3_s1_os_iface.get_os_interface()} {s3_s2_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'")
    print("stdout: {}".format(stdout))
    
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

stdout: ('', '')


### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [131]:
for switch_name in [s1_name, s2_name, s3_name]:
    switch_node = slice.get_node(name=switch_name)
    management_ip_switch = str(switch_node.get_management_ip())
    print("Swtitch Name        : {}".format(switch_node.get_name()))
    print("Management IP    : {}".format(management_ip_switch))
    
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}_commands.txt'
    print(cmd_file)
    file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))

    #stdout = execute_script(username, switch_node, f"sudo sh -c 'cat {cmd_file} | docker exec -it fabric_p4 simple_switch_CLI  > /tmp/script.log 2>&1'")
    stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

Swtitch Name        : s1
Management IP    : 63.239.135.76
s1_commands.txt
file_attributes: -rw-rw-r--   1 1000     1000          185 20 Jan 12:30 ?
stdout: ('Obtaining JSON from switch...\nDone\nControl utility for runtime P4 table manipulation\nRuntimeCmd: Setting default action of myTunnel_exact\naction:              drop\nruntime data:        \nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:01\naction:              myTunnel_forward\nruntime data:        00:01\nEntry has been added with handle 0\nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:02\naction:              myTunnel_forward\nruntime data:        00:02\nEntry has been added with handle 1\nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:03\naction:              myTunnel_forward\nruntime data:        00:03\nEntry has been added with handle 2\nRuntimeCmd: \n', '')
Swtitch Name        : s2
Managem

## The switches are now configured and running. Now we are going to send packets over the switches.

We are going to use `send.py` and `receive.py`. We are going to re-upload them to the servers and use them. Make sure to modify the interface names in the script accordingly.

In [136]:
h1.upload_file('scripts/send.py', 'send.py')

<SFTPAttributes: [ size=1744 uid=1000 gid=1000 mode=0o100664 atime=1642750892 mtime=1642751212 ]>

In [133]:
h2.upload_file('scripts/receive.py', 'receive.py')

<SFTPAttributes: [ size=1137 uid=1000 gid=1000 mode=0o100664 atime=1642750933 mtime=1642750933 ]>

In [137]:
h1.execute('sudo mv send.py tutorials/exercises/basic_tunnel/send_modified.py')

('', '')

In [138]:
h2.execute('sudo mv receive.py tutorials/exercises/basic_tunnel/receive_modified.py')

('', '')

## Below, we send a few packets.

In [146]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 2 10.10.2.2 "message100"\'')

('', '')

## And below we receive them.

In [147]:
h2.execute('timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')

("sniffing on ens7\ngot a packet\n###[ Ethernet ]### \n  dst       = ff:ff:ff:ff:ff:ff\n  src       = 16:55:bb:75:01:1a\n  type      = 0x1212\n###[ MyTunnel ]### \n     pid       = 2048\n     dst_id    = 2\n###[ IP ]### \n        version   = 4\n        ihl       = 5\n        tos       = 0x0\n        len       = 30\n        id        = 1\n        flags     = \n        frag      = 0\n        ttl       = 64\n        proto     = hopopt\n        chksum    = 0x6091\n        src       = 10.20.4.47\n        dst       = 10.10.2.2\n        \\options   \\\n###[ Raw ]### \n           load      = 'message100'\n###[ Padding ]### \n              load      = '\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'\n\ngot a packet\n###[ Ethernet ]### \n  dst       = ff:ff:ff:ff:ff:ff\n  src       = 16:55:bb:75:01:1a\n  type      = 0x1212\n###[ MyTunnel ]### \n     pid       = 2048\n     dst_id    = 2\n###[ IP ]### \n        version   = 4\n        ihl       = 5\n        tos       = 0x0\n        len       = 30\n      

## Delete Slice

In [96]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()